In [1]:
import sys
from pathlib import Path

In [2]:
import ethicml
from ethicml.algorithms.inprocess import GPyT, GPyTDemPar, GPyTEqOdds, LR, SVM, Agarwal, Kamiran, Kamishima, LR
from ethicml.evaluators import evaluate_models, CrossValidator, run_metrics
from ethicml.data import Compas, Adult, load_data
from ethicml.metrics import Accuracy, ProbPos, TPR, TNR, AbsCV
from ethicml.preprocessing import train_test_split
from ethicml.visualisation.plot import plot_mean_std_box

In [3]:
# YOU DON'T HAVE TO UNDERSTAND THE CODE IN THIS CELL
# we only tell the model where the "run.py" is and where the python executable is
file_name = Path('..') / "run.py"
def gp(**kwargs):
    return GPyT(file_name=file_name, executable=sys.executable, **kwargs)
def gp_dp(**kwargs):
    return GPyTDemPar(file_name=file_name, executable=sys.executable, **kwargs)
def gp_eo(**kwargs):
    return GPyTEqOdds(file_name=file_name, executable=sys.executable, **kwargs)

In [4]:
# tnr_race_False = 0.724
# tnr_race_True = 0.702
# tnr_sex_True = 0.724
# tnr_sex_False = 0.744
tnr_in_true_race = 0.71
tnr_in_false_race = 0.74
tnr_in_true_sex = 0.72
tnr_in_false_sex = 0.77

In [9]:
algos = []

algos += [gp(epochs=70, s_as_input=True)]
algos += [gp(epochs=70, s_as_input=False)]
# algos += [gp_dp(epochs=70, s_as_input=True)]
for tpr in [0.7]:
    algos += [gp_eo(epochs=70, s_as_input=True, tnr1=tnr_in_true_race, tnr0=tnr_in_true_race, tpr0=tpr, tpr1=tpr)]
    algos += [gp_eo(epochs=70, s_as_input=False, tnr1=tnr_in_false_race, tnr0=tnr_in_false_race, tpr0=tpr, tpr1=tpr)]

baselines = [
    LR(),
    SVM(),
    Agarwal(fairness="EqOd"),
    Kamiran(),
#     Kamishima(),
]
algos += baselines

In [10]:
data = [
    Compas("Race"),
#     Compas("Sex"),
#     Adult("Race"),
#     Adult("Sex"),
]

In [11]:
results = evaluate_models(
    datasets=data,
    inprocess_models=algos,
    metrics=[Accuracy(), ProbPos(), TPR(), TNR()],
    per_sens_metrics=[ProbPos(), TPR(), TNR()], 
    repeats=5,
    delete_prev=True,  # delete previous results
)
results

 15%|█▌        | 6/40 [01:33<07:29, 13.22s/it, model=Agarwal LR, dataset=Compas Race, transform=no_transform, repeat=0]                                                 /mnt/archive/fairness/py_env/lib/python3.6/site-packages/fairlearn/classred.py:309: FutureWarning: The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
  if not Qsum.index.contains(h_idx):
/mnt/archive/fairness/py_env/lib/python3.6/site-packages/fairlearn/classred.py:309: FutureWarning: The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
  if not Qsum.index.contains(h_idx):
/mnt/archive/fairness/py_env/lib/python3.6/site-packages/fairlearn/classred.py:309: FutureWarning: The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
  if not Qsum.index.contains(h_idx):
/mnt/archive/fairness/py_env/lib/pyth

Accuracy  \
dataset     transform    model                                              repeat              
Compas Race no_transform GPyT_in_True                                       0-2410   0.679092   
                         GPyT_in_False                                      0-2410   0.679903   
                         GPyT_eq_odds_in_True_0tnr_0.71_1tnr_0.71_0tpr_0... 0-2410   0.670178   
                         GPyT_eq_odds_in_False_0tnr_0.74_1tnr_0.74_0tpr_... 0-2410   0.682334   
                         Logistic Regression                                0-2410   0.675851   
                         SVM                                                0-2410   0.687196   
                         Agarwal LR                                         0-2410   0.672609   
                         Kamiran & Calders LR                               0-2410   0.676661   
                         GPyT_in_True                                       1-4820   0.687196   
                         GPyT_in_False                                      1-4820   0.692869   
                         GPyT_eq_odds_in_True_0tnr_0.71_1tnr_0.71_0tpr_0... 1-4820   0.677472   
                         GPyT_eq_odds_in_False_0tnr_0.74_1tnr_0.74_0tpr_... 1-4820   0.683955   
                         Logistic Regression                                1-4820   0.682334   
                         SVM                                                1-4820   0.681524   
                         Agarwal LR                                         1-4820   0.654781   
                         Kamiran & Calders LR                               1-4820   0.681524   
                         GPyT_in_True                                       2-7230   0.683144   
                         GPyT_in_False                                      2-7230   0.687196   
                         GPyT_eq_odds_in_True_0tnr_0.71_1tnr_0.71_0tpr_0... 2-7230   0.679903   
                         GPyT_eq_odds_in_False_0tnr_0.74_1tnr_0.74_0tpr_... 2-7230   0.674230   
                         Logistic Regression                                2-7230   0.668558   
                         SVM                                                2-7230   0.673420   
                         Agarwal LR                                         2-7230   0.667747   
                         Kamiran & Calders LR                               2-7230   0.665316   
                         GPyT_in_True                                       3-9640   0.700972   
                         GPyT_in_False                                      3-9640   0.698541   
                         GPyT_eq_odds_in_True_0tnr_0.71_1tnr_0.71_0tpr_0... 3-9640   0.690438   
                         GPyT_eq_odds_in_False_0tnr_0.74_1tnr_0.74_0tpr_... 3-9640   0.696921   
                         Logistic Regression                                3-9640   0.696921   
                         SVM                                                3-9640   0.689627   
                         Agarwal LR                                         3-9640   0.685575   
                         Kamiran & Calders LR                               3-9640   0.688006   
                         GPyT_in_True                                       4-12050  0.692058   
                         GPyT_in_False                                      4-12050  0.697731   
                         GPyT_eq_odds_in_True_0tnr_0.71_1tnr_0.71_0tpr_0... 4-12050  0.684765   
                         GPyT_eq_odds_in_False_0tnr_0.74_1tnr_0.74_0tpr_... 4-12050  0.700162   
                         Logistic Regression                                4-12050  0.678282   
                         SVM                                                4-12050  0.690438   
                         Agarwal LR                                         4-12050  0.674230   
                         Kamiran & Calders LR                               4-12050  0.672609   

          

In [12]:
plot_mean_std_box(results, Accuracy(), TPR())

In [3]:
train, test = train_test_split(load_data(Adult()))

In [4]:
# Fair grid search
primary = Accuracy()
fair_measure = AbsCV()
hyperparams = dict(C=[1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7])
lr_cv = CrossValidator(LR, hyperparams, folds=5)
lr_cv.run(train, measures=[primary, fair_measure])
lr_cv.best_hyper_params(primary)

fold_0_model_Logistic Regression, C=1_completed
fold_0_model_Logistic Regression, C=0.1_completed
fold_0_model_Logistic Regression, C=0.01_completed
fold_0_model_Logistic Regression, C=0.001_completed
fold_0_model_Logistic Regression, C=0.0001_completed
fold_0_model_Logistic Regression, C=1e-05_completed
fold_0_model_Logistic Regression, C=1e-06_completed
fold_0_model_Logistic Regression, C=1e-07_completed
fold_1_model_Logistic Regression, C=1_completed
fold_1_model_Logistic Regression, C=0.1_completed
fold_1_model_Logistic Regression, C=0.01_completed
fold_1_model_Logistic Regression, C=0.001_completed
fold_1_model_Logistic Regression, C=0.0001_completed
fold_1_model_Logistic Regression, C=1e-05_completed
fold_1_model_Logistic Regression, C=1e-06_completed
fold_1_model_Logistic Regression, C=1e-07_completed
fold_2_model_Logistic Regression, C=1_completed
fold_2_model_Logistic Regression, C=0.1_completed
fold_2_model_Logistic Regression, C=0.01_completed
fold_2_model_Logistic Regressio

{'C': 0.1}

In [5]:
lr_best_acc = lr_cv.results.get_best_result(primary)
lr_best_fair = lr_cv.results.get_best_in_top_k(primary, fair_measure, top_k=3)
print("best accuracy:", lr_best_acc)
print("best fair(+accuracy):", lr_best_fair)

best accuracy: ResultTuple(params={'C': 0.1}, fold_id=-1, scores={'Accuracy': 0.8460072554374714, 'CV absolute': 0.825007917407934})
best fair(+accuracy): ResultTuple(params={'C': 1e-06}, fold_id=-1, scores={'Accuracy': 0.7909169578659723, 'CV absolute': 0.9530144052067022})


In [6]:
lr_best_acc_model = LR(**lr_best_acc.params)
results = evaluate_models(
    datasets=[Adult()],
    inprocess_models=[LR(**lr_best_fair.params), lr_best_acc_model],
    metrics=[Accuracy(), ProbPos(), TPR(), TNR()],
    per_sens_metrics=[ProbPos(), TPR(), TNR()], 
    repeats=3,
    delete_prev=True,  # delete previous results
)
results

100%|██████████| 6/6 [00:03<00:00,  1.84it/s, model=Logistic Regression, C=0.1, dataset=Adult Sex, transform=no_transform, repeat=2]  


Accuracy  \
dataset   transform    model                        repeat             
Adult Sex no_transform Logistic Regression, C=1e-06 0-2410  0.789829   
                       Logistic Regression, C=0.1   0-2410  0.851852   
                       Logistic Regression, C=1e-06 1-4820  0.788281   
                       Logistic Regression, C=0.1   1-4820  0.843339   
                       Logistic Regression, C=1e-06 2-7230  0.793367   
                       Logistic Regression, C=0.1   2-7230  0.849972   

                                                                 TNR  \
dataset   transform    model                        repeat             
Adult Sex no_transform Logistic Regression, C=1e-06 0-2410  0.962122   
                       Logistic Regression, C=0.1   0-2410  0.930140   
                       Logistic Regression, C=1e-06 1-4820  0.961874   
                       Logistic Regression, C=0.1   1-4820  0.935422   
                       Logistic Regression, C=1e-06 2-7230  0.964702   
                       Logistic Regression, C=0.1   2-7230  0.934722   

                                                            TNR_sex_Male_0  \
dataset   transform    model                        repeat                   
Adult Sex no_transform Logistic Regression, C=1e-06 0-2410        0.970254   
                       Logistic Regression, C=0.1   0-2410        0.974168   
                       Logistic Regression, C=1e-06 1-4820        0.967232   
                       Logistic Regression, C=0.1   1-4820        0.982652   
                       Logistic Regression, C=1e-06 2-7230        0.967868   
                       Logistic Regression, C=0.1   2-7230        0.978056   

                                                            TNR_sex_Male_0-sex_Male_1  \
dataset   transform    model                        repeat                              
Adult Sex no_transform Logistic Regression, C=1e-06 0-2410                   0.013044   
                       Logistic Regression, C=0.1   0-2410                   0.070622   
                       Logistic Regression, C=1e-06 1-4820                   0.008689   
                       Logistic Regression, C=0.1   1-4820                   0.076589   
                       Logistic Regression, C=1e-06 2-7230                   0.005081   
                       Logistic Regression, C=0.1   2-7230                   0.069547   

                                                            TNR_sex_Male_0/sex_Male_1  \
dataset   transform    model                        repeat                              
Adult Sex no_transform Logistic Regression, C=1e-06 0-2410                   0.986556   
                       Logistic Regression, C=0.1   0-2410                   0.927505   
                       Logistic Regression, C=1e-06 1-4820                   0.991017   
                       Logistic Regression, C=0.1   1-4820                   0.922058   
                       Logistic Regression, C=1e-06 2-7230                   0.994750   
                       Logistic Regression, C=0.1   2-7230                   0.928892   

                                                            TNR_sex_Male_1  \
dataset   transform    model                        repeat                   
Adult Sex no_transform Logistic Regression, C=1e-06 0-2410        0.957210   
                       Logistic Regression, C=0.1   0-2410        0.903546   
                       Logistic Regression, C=1e-06 1-4820        0.958543   
                       Logistic Regression, C=0.1   1-4820        0.906063   
                       Logistic Regression, C=1e-06 2-7230        0.962787   
                       Logistic Regression, C=0.1   2-7230        0.908509   

                                                                 TPR  \
dataset   transform    model                        repeat             
Adult Sex no_transform Logistic Regression, C=1e-06 0-2410  0.272566   
              

In [7]:
plot_mean_std_box(results, Accuracy(), ProbPos())